In [124]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [125]:
from planner import JourneyPlanner
from collections import defaultdict
import pickle

# Loading timetable

In [87]:
with open('../data/footpath.pickle', 'rb') as f:
    footpaths = pickle.load(f)

In [88]:
with open('../data/trips.pickle', 'rb') as f:
    trips = pickle.load(f)

In [89]:
with open('../data/stops.pickle', 'rb') as f:
    stops = pickle.load(f)

In [90]:
with open('../data/connections_data.pickle', 'rb') as f:
    connections = pickle.load(f)

In [722]:
paper_conn = [[
    ("y", "t", 100000, 110000, "1"),
    ("z", "t", 90000, 120000, "2"),
    ("x", "t", 80000, 130000, "3"),
    ("x", "y", 80000, 90000, "4"),
    ("s", "z", 70000, 80000, "5"),
    ("s", "x", 60000, 70000, "6"),
    ("s", "t", 50000, 140000, "7")
]]
timetable = (None, paper_conn, None, defaultdict(lambda: list()))

In [91]:
timetable = stops, connections, trips, footpaths

# Plan a route

In [4]:
planner = JourneyPlanner(timetable)

journeys = planner.plan_route(2, "8503315:0:2", "8503380", 1653491740, 1653501740, max_changes = 10)

NameError: name 'JourneyPlanner' is not defined

In [3]:
len(journeys)

NameError: name 'journeys' is not defined

In [106]:
def search_station_id(name):
    for stop in stops:
        if name in stops[stop]['stop_name']:
            print(stop)

In [107]:
search_station_id("Winterberg ZH, Wältiwis")

8575942


In [67]:
stops['8503380']

{'stop_name': 'Winterberg ZH, Dorf',
 'stop_lat': '47.4564062782486',
 'stop_lon': '8.69305988770458',
 'parent_station': ''}

In [62]:
stops['8575942']

{'stop_name': 'Winterberg ZH, Wältiwis',
 'stop_lat': '47.4546690948399',
 'stop_lon': '8.69626687326933',
 'parent_station': ''}